In [1]:
%pip install pandas
import pandas as pd
%pip install numpy
import re
%pip install nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

You should consider upgrading via the 'c:\Users\cedri\Desktop\Datasets and Artifact\env\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\cedri\Desktop\Datasets and Artifact\env\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\Users\cedri\Desktop\Datasets and Artifact\env\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Read the CSV files (Add the path to the CSV files)
billboard_lyrics_spotify = pd.read_csv('C:/Users/cedri/Desktop/Datasets and Artifact/data/billboard-lyrics-spotify.csv')


In [5]:
column_headers = billboard_lyrics_spotify.columns
print(column_headers)

Index(['artist_all', 'artist_base', 'rank', 'song', 'year', 'artist_featured',
       'song_clean', 'artist_clean', 'lyrics', 'acousticness', 'danceability',
       'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'release_date', 'speechiness', 'tempo',
       'time_signature', 'valence', 'duration_min', 'num_words',
       'words_per_sec', 'num_uniq_words', 'decade', 'uniq_ratio'],
      dtype='object')


In [7]:
print("\nBillboard Lyrics Spotify Data:")
print(billboard_lyrics_spotify.head(1))



Billboard Lyrics Spotify Data:
    artist_all  artist_base  rank                       song  year  \
0  percy faith  percy faith     1  theme from a summer place  1960   

  artist_featured                 song_clean artist_clean  \
0             NaN  theme from a summer place  percy faith   

                                              lyrics  acousticness  ...  \
0  theres a summer place where it may rain or sto...         0.631  ...   

   speechiness   tempo  time_signature  valence  duration_min  num_words  \
0       0.0253  92.631             4.0    0.749      2.414883      104.0   

   words_per_sec  num_uniq_words  decade uniq_ratio  
0       0.717771            58.0    1960   1.793103  

[1 rows x 30 columns]


In [8]:
# Check for duplicates in the billboard_lyrics_spotify dataframe
billboard_lyrics_spotify_duplicates = billboard_lyrics_spotify.duplicated().sum()
print(f"Number of duplicate rows in billboard_lyrics_spotify dataframe: {billboard_lyrics_spotify_duplicates}")

# Check for missing data in the billboard_lyrics_spotify dataframe
billboard_lyrics_spotify_missing = billboard_lyrics_spotify.isnull().sum()
print(f"Number of missing values in billboard_lyrics_spotify dataframe: \n{billboard_lyrics_spotify_missing}")


Number of duplicate rows in billboard_lyrics_spotify dataframe: 0
Number of missing values in billboard_lyrics_spotify dataframe: 
artist_all             0
artist_base            0
rank                   0
song                   0
year                   0
artist_featured     4993
song_clean             0
artist_clean           0
lyrics                50
acousticness         170
danceability         170
energy               170
explicit             170
instrumentalness     170
key                  170
liveness             170
loudness             170
mode                 170
popularity           170
release_date         170
speechiness          170
tempo                170
time_signature       170
valence              170
duration_min         170
num_words             50
words_per_sec        213
num_uniq_words        50
decade                 0
uniq_ratio            74
dtype: int64


The missing values of artist_featured, which suggests that not all songs have featured artists. This is expected as not all songs have features artists

In [9]:
# Fill missing values in the 'artist_featured' column of the 'billboard_lyrics_spotify' DataFrame with the string 'No Featured Artist'
billboard_lyrics_spotify['artist_featured'] = billboard_lyrics_spotify['artist_featured'].fillna('No Featured Artist')

# Print the number of missing values in the 'artist_featured' column of the 'billboard_lyrics_spotify' DataFrame
print("Missing values in billboard_lyrics_spotify:", billboard_lyrics_spotify['artist_featured'].isnull().sum())

Missing values in billboard_lyrics_spotify: 0


Missing Lyrics and spotify meta data values. 

The `billboard_lyrics_spotify` DataFrame has 5566 rows of data. The `billboard_lyrics_spotify_missing` variable shows that there are 50 songs that have missing lyrics. 

So, there is a small percentage of songs in DataFrames that do not have lyrics.

In [10]:
# Drop any rows with missing lyrics
billboard_lyrics_spotify = billboard_lyrics_spotify.dropna(subset=['lyrics'])

# Calculate the number of songs with missing lyrics after dropping
missing_lyrics = billboard_lyrics_spotify['lyrics'].isnull().sum()

# Print the number of songs with missing lyrics
print(f"Number of songs with missing lyrics: {missing_lyrics}")


Number of songs with missing lyrics: 0


In [11]:
# Drop the rows with missing data
billboard_lyrics_spotify = billboard_lyrics_spotify.dropna()

print("Total available songs: ", billboard_lyrics_spotify.shape[0])



Total available songs:  5330


In [12]:
# Function to preprocess lyrics
def preprocess_lyrics(lyrics):
    # Lowercase the lyrics (Normalization)
    lyrics = lyrics.lower()
    
    # Remove special characters
    lyrics = re.sub(r'\W', ' ', lyrics)
    
    # Remove single characters
    lyrics = re.sub(r'\s+[a-zA-Z]\s+', ' ', lyrics)
    
    # Remove single characters from the start
    lyrics = re.sub(r'\^[a-zA-Z]\s+', ' ', lyrics) 
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(lyrics)
    lyrics = [i for i in tokens if not i in stop_words]
    
    # Stemming
    ps = PorterStemmer()
    lyrics = [ps.stem(word) for word in lyrics]
    
    return lyrics

In [13]:
# Create a dataset without null values
df_no_nulls = billboard_lyrics_spotify.dropna(subset=['lyrics'])

In [14]:
from transformers import BertModel, BertTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

c:\Users\cedri\Desktop\Datasets and Artifact\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
def vectorize_text(text):
    # Join the preprocessed words back into a string
    text = ' '.join(text)

    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Get the embeddings
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    # Average the embeddings
    vector = torch.mean(embeddings, dim=1)

    return vector.numpy()



In [16]:
# Apply the function to your preprocessed lyrics data
df_no_nulls['bert_vector'] = df_no_nulls['lyrics'].apply(vectorize_text)

In [12]:
# Save the DataFrame without null values to a CSV file
df_no_nulls.to_csv('df_no_nulls.csv', index=False)